# 1. Imports and reading data

In [25]:
%%time
import time
import re
import pandas as pd
import numpy as np
import umap
import string
import hdbscan
import spacy
import gensim
import sklearn

from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT

nlp = spacy.load('en_core_web_sm', disable = ['parser', 'ner'])
stop = stopwords.words('english')
model=SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')
kw_extractor = KeyBERT(model=model)

%config Completer.use_jedi = False

Wall time: 15.3 s


In [26]:
# xlm-r-distilroberta-base-paraphrase-v1
# distilbert-base-nli-mean-tokens

In [27]:
start = time.time()

In [28]:
df = pd.read_csv('hsbc_comments.csv')
df.rename(columns={"message":"comments"},inplace=True)

# 2. Preprocessing

In [29]:
#This takes each sentence as inputs and returns list of clean tokens for that sentence
def TextCleaner(doc):
    doc = re.sub("[,.']", "", str(doc))
    doc = [token for token in doc.lower().split(' ') if len(re.sub("[a-z]", "", token)) <= 0]
    doc = nlp(" ".join(doc))
    taglist = ['RB','RBR', 'RBS','JJR','JJ','JJS','NN','NNS','VB','VBG','VBP','VBN']
    poslist = ['ADJ','ADV','NOUN','VERB']
    doc = [token.lemma_ for token in doc if token.tag_ in taglist and token.pos_ in poslist]
    doc = [token for token in doc if not token in stop and len(token)>2 and len(token)<15]
    return doc

In [30]:
# This function converts a list of clean documents to trigrams 
def TextNGram(doc,ngrams = 3):
    bigram = gensim.models.Phrases(doc , min_count=7, threshold=70) # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    if ngrams == 2:
        doc = [bigram_mod[sent] for sent in doc]
        return doc
    elif ngrams == 3:
        trigram = gensim.models.Phrases(bigram[doc], threshold=5)
        trigram_mod = gensim.models.phrases.Phraser(trigram)
        doc = [trigram_mod[bigram_mod[sent]] for sent in doc]
        return doc
    else:
        return doc
        

In [31]:
%%time
#First 
df['Processed'] = [TextCleaner(f) for f in df['comments']]
#Second
df['Processed_ngram'] = TextNGram(df['Processed'])

Wall time: 6.4 s


In [32]:
print(df['Processed_ngram'].tolist())

[['like', 'say', 'massive', 'thank', 'hsbc', 'telephone', 'use', 'card', 'help', 'thorough', 'process', 'stop', 'friendly', 'always', 'bank', 'hsbc'], ['run_try', 'get_basic', 'banking', 'service_right', 'first_try', 'get', 'money_transfer', 'credit_card', 'account', 'back_current', 'account_refund', 'british_january', 'initially_tell', 'call_christian', 'take_day', 'still_sign', 'use_web', 'chat_service', 'today', 'tell_today', 'tuesday_january', 'call_askshith', 'take_working', 'day_january', 'hour', 'get', 'information_today', 'keep_get', 'run_around', 'people_overseas', 'apologise_patronising', 'way', 'tell', 'ever_apologise', 'ever_also', 'try_blame', 'somewhere_never', 'fault_restore', 'confidence_bank'], ['send', 'neccesary', 'number', 'use', 'online', 'service', 'wait', 'week', 'help'], ['past', 'hour', 'wait', 'time', 'service'], ['melanie', 'humberstone'], ['brilliant'], ['absolutely', 'fume', 'sit', 'hold', 'almost', 'hour', 'telephone', 'line', 'able', 'change', 'withdrawal

In [33]:
data = pd.Series(df['Processed_ngram'].apply(lambda x: " ".join(x) if len(x) > 4 else np.nan))

In [34]:
data.dropna(axis=0, inplace=True)
data.reset_index(drop=True,inplace=True)

In [35]:
list_data = data.unique().tolist()

# 3. Model, Dimension Reduction and Saving Model

##     a. BERT Model

In [36]:
%%time
embeddings = model.encode(list_data, show_progress_bar=True)

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Wall time: 56 s


## b. UMAP Dimension Reduction

In [37]:
embedding_norm = sklearn.preprocessing.normalize(embeddings, norm='l2')

In [38]:
%%time
best_model = umap.UMAP(n_components=5,min_dist=0.0).fit_transform(embedding_norm)

# #n_neighbors=10, min_dist=0.0, 

Wall time: 10.8 s


# 4. Clustering and Reducing Clusters with Cosine-Similarity

## a. HDBSCAN Clustering

In [39]:
%%time
cluster = KMeans(n_clusters=4, init="k-means++").fit_predict(best_model)

Wall time: 75 ms


In [40]:
%%time
docs = pd.DataFrame(list_data,columns=["comments"])
docs["cluster"] = cluster
labeled_docs = docs.groupby(["cluster"], as_index=False).agg({"comments": " ".join})
array_text = labeled_docs.comments.tolist()

Wall time: 13 ms


In [41]:
%%time
for j in range(len(array_text)):
    keywords = kw_extractor.extract_keywords(array_text[j], top_n=10)
    print(f'\033[1m   Cluster {j+1}: \033[0m')
    print([word for word, degree in keywords])

   Cluster 1: 
['bad_service', 'transfer_account', 'bankrupt', 'bank', 'problem_error', 'transfer_money', 'customer_service', 'business_account', 'bad_team', 'banking']
   Cluster 2: 
['try_get', 'run_try', 'chat_service', 'call_christian', 'transfer_account', 'customer_service', 'try', 'call_barclaycard', 'telephone_banking', 'account_refund']
   Cluster 3: 
['staff_pensioner', 'pensioner', 'fraud', 'pension', 'state_pension', 'brazil', 'trustee', 'financial', 'clawbank', 'claw_back']
   Cluster 4: 
['account_refund', 'saving_account', 'fraud', 'secure_key', 'safeguard', 'telephone_banking', 'bank', 'letter_say', 'safeguarding', 'credit_card']
Wall time: 45.6 s


In [42]:
end = time.time()
print(end - start)

213.5800621509552
